In [1]:
import pandas as pd
import cv2
import os
import ntpath
import math

In [2]:
# Leo el pandas
df = pd.read_csv('processed_3/data.csv')

In [3]:
df.head()

,user_ID,frame_ID,checkpt,BB_x,BB_y,BB_w,BB_h,vid_ID,groundtruth,finish
0,1,/media/david/Seagate Basic/Datasets/TGC2020/TG...,Ayagaures,NaN,NaN,NaN,NaN,/media/david/Seagate Basic/Datasets/TGC2020/pr...,0,1
1,1,/media/david/Seagate Basic/Datasets/TGC2020/TG...,Ayagaures,NaN,NaN,NaN,NaN,/media/david/Seagate Basic/Datasets/TGC2020/pr...,0,1
2,1,/media/david/Seagate Basic/Datasets/TGC2020/TG...,Ayagaures,NaN,NaN,NaN,NaN,/media/david/Seagate Basic/Datasets/TGC2020/pr...,0,1
3,1,/media/david/Seagate Basic/Datasets/TGC2020/TG...,Ayagaures,NaN,NaN,NaN,NaN,/media/david/Seagate Basic/Datasets/TGC2020/pr...,0,1
4,1,/media/david/Seagate Basic/Datasets/TGC2020/TG...,Ayagaures,NaN,NaN,NaN,NaN,/media/david/Seagate Basic/Datasets/TGC2020/pr...,0,1


In [11]:
# Clase para leer

class Read_Video():
    def __init__(self):
        pass
    
    def _frame_from_video(self, video):
        while video.isOpened():
            success, frame = video.read()
            if success:
                yield frame
            else:
                break
    
    def print_bb_on_image(self, df, input_img_path): # NO PROBADA, PENDIENTE DE PROBAR
        # necesitamos una column con el nombre base de cada imagen limpio
        df['img_base'] = df.frame_ID.str.split('/').str[-1]
        # quito el path de la ruta de la imagen
        input_img = ntpath.basename(input_img)
        # obtengo la informacion de ese frame
        frame_info = df[df.img_base==input_img]
        # cargo la imagen
        img = cv2.imread(input_img_path)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        bbox = [int(frame_info.BB_x),
                int(frame_info.BB_y),
                int(frame_info.BB_x)+int(frame_info.BB_w),
                int(frame_info.BB_y)+int(frame_info.BB_h)]
        img = cv2.rectangle(img,
                            (bbox[0], bbox[1]),
                            (bbox[2], bbox[3]),
                            color,
                            2)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img
    
    def print_bb_on_video(self, df, in_video, out_video):
        user = int(ntpath.basename(in_video).split('.')[0].split('_')[0])
        place = ntpath.basename(in_video).split('.')[0].split('_')[1]

        vid_df = df[((df.user_ID==user)&(df.checkpt==place))]

        
        try:
            video = cv2.VideoCapture(in_video)
            width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
            frames_per_second = video.get(cv2.CAP_PROP_FPS)

            output_file = cv2.VideoWriter(
                filename=out_video,
                fourcc=cv2.VideoWriter_fourcc(*'mp4v'),
                fps=float(frames_per_second),
                frameSize=(width, height),
                isColor=True,
            )

            frame_gen = self._frame_from_video(video)
    
            for id_frame, frame in enumerate(frame_gen):
                frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
                print('Procesando frame... ', id_frame)
                frame_info = vid_df.iloc[id_frame]
                if not(math.isnan(frame_info.BB_x)):
                    color= (255, 0, 0)
                    bbox = [int(frame_info.BB_x),
                            int(frame_info.BB_y),
                            int(frame_info.BB_x)+int(frame_info.BB_w),
                            int(frame_info.BB_y)+int(frame_info.BB_h)]
                    frame = cv2.rectangle(frame,
                                          (bbox[0], bbox[1]),
                                          (bbox[2], bbox[3]),
                                          color,
                                          2)
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    output_file.write(frame)
        except ValueError:
            print("Oops!  Problem.  Try again...")

        output_file.release()
        video.release()
    
    def get_bbs(self, df, user, checkpoint):
        
        vid_df = df[((df.user_ID==user)&(df.checkpt==checkpoint))]
        
        output = []
        
        for id_frame in range(0,len(vid_df)):
            frame_info = vid_df.iloc[id_frame]
            if not(math.isnan(frame_info.BB_x)):
                bbox = [int(frame_info.BB_x),
                        int(frame_info.BB_y),
                        int(frame_info.BB_x)+int(frame_info.BB_w),
                        int(frame_info.BB_y)+int(frame_info.BB_h)]
            
                out_dict = {}
                out_dict['Frame'] = id_frame
                out_dict['BB'] = bbox
            
                output.append(out_dict)
        return output

In [12]:
# Invoco clase
reader = Read_Video()

In [9]:
# Pruebo con un video:

reader.print_bb_on_video(df, 'tmp_data/4_PresadeHornos.mp4', 'prueba.mp4')

Procesando frame...  0
Procesando frame...  1
Procesando frame...  2
Procesando frame...  3
Procesando frame...  4
Procesando frame...  5
Procesando frame...  6
Procesando frame...  7
Procesando frame...  8
Procesando frame...  9
Procesando frame...  10
Procesando frame...  11
Procesando frame...  12
Procesando frame...  13
Procesando frame...  14
Procesando frame...  15
Procesando frame...  16
Procesando frame...  17
Procesando frame...  18
Procesando frame...  19
Procesando frame...  20
Procesando frame...  21
Procesando frame...  22
Procesando frame...  23
Procesando frame...  24
Procesando frame...  25
Procesando frame...  26
Procesando frame...  27
Procesando frame...  28
Procesando frame...  29
Procesando frame...  30
Procesando frame...  31
Procesando frame...  32
Procesando frame...  33
Procesando frame...  34
Procesando frame...  35
Procesando frame...  36
Procesando frame...  37
Procesando frame...  38
Procesando frame...  39
Procesando frame...  40
Procesando frame...  41
Pr

In [13]:
# Pruebo extrayendo los puntos con deteccion en una list dict

reader.get_bbs(df, 1, 'Ayagaures')

[{'Frame': 20, 'BB': [872, 233, 930, 524]},
 {'Frame': 21, 'BB': [868, 226, 926, 515]},
 {'Frame': 22, 'BB': [864, 221, 923, 513]},
 {'Frame': 23, 'BB': [847, 208, 912, 521]},
 {'Frame': 24, 'BB': [841, 204, 909, 528]},
 {'Frame': 25, 'BB': [839, 205, 908, 528]},
 {'Frame': 26, 'BB': [834, 210, 905, 545]},
 {'Frame': 27, 'BB': [827, 217, 899, 553]},
 {'Frame': 28, 'BB': [820, 222, 892, 553]},
 {'Frame': 29, 'BB': [809, 219, 884, 550]},
 {'Frame': 30, 'BB': [797, 214, 877, 560]},
 {'Frame': 31, 'BB': [783, 207, 871, 572]},
 {'Frame': 32, 'BB': [772, 206, 868, 591]},
 {'Frame': 33, 'BB': [754, 206, 857, 603]},
 {'Frame': 34, 'BB': [738, 215, 845, 614]},
 {'Frame': 35, 'BB': [727, 222, 838, 620]},
 {'Frame': 36, 'BB': [716, 227, 831, 623]},
 {'Frame': 37, 'BB': [701, 220, 820, 624]},
 {'Frame': 38, 'BB': [686, 207, 811, 625]},
 {'Frame': 39, 'BB': [668, 195, 800, 630]},
 {'Frame': 40, 'BB': [652, 192, 793, 650]},
 {'Frame': 41, 'BB': [632, 174, 787, 676]},
 {'Frame': 42, 'BB': [611, 170, 